In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
URL = "https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030-2/" 
r = requests.get(URL) 
print(r.text) 

<!doctype html >
<!--[if IE 8]>    <html class="ie8" lang="en"> <![endif]-->
<!--[if IE 9]>    <html class="ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-US"> <!--<![endif]-->
<head>
    <title>Rise of e-health and its impact on humans by the year 2030 - Blackcoffer Insights</title>
    <meta charset="UTF-8" />
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <link rel="pingback" href="https://insights.blackcoffer.com/xmlrpc.php" />
    <meta name='robots' content='index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1' />

	<!-- This site is optimized with the Yoast SEO plugin v21.1 - https://yoast.com/wordpress/plugins/seo/ -->
	<meta name="description" content="Being connected to the internet and being made available to all also attracts a lot of nefarious elements, e-Health seems like the probable future for the entire healthcare industry during the Lockdown period" />
	<link rel="canonical" href="https://insi

In [2]:
soup = BeautifulSoup(r.content, 'html5lib') # If this line causes an error, run 'pip install html5lib' or install html5lib 
print(soup.prettify()) 

<!DOCTYPE html>
<!--[if IE 8]>    <html class="ie8" lang="en"> <![endif]-->
<!--[if IE 9]>    <html class="ie9" lang="en"> <![endif]-->
<!--[if gt IE 8]><!-->
<html lang="en-US">
 <!--<![endif]-->
 <head>
  <title>
   Rise of e-health and its impact on humans by the year 2030 - Blackcoffer Insights
  </title>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://insights.blackcoffer.com/xmlrpc.php" rel="pingback"/>
  <meta content="index, follow, max-image-preview:large, max-snippet:-1, max-video-preview:-1" name="robots"/>
  <!-- This site is optimized with the Yoast SEO plugin v21.1 - https://yoast.com/wordpress/plugins/seo/ -->
  <meta content="Being connected to the internet and being made available to all also attracts a lot of nefarious elements, e-Health seems like the probable future for the entire healthcare industry during the Lockdown period" name="description"/>
  <link href="https://insights.blackcoffer.co

In [7]:
soup.find('div', class_="tdb-block-inner td-fix-index").text

''

In [8]:
soup.find('title').text.strip()

'Rise of e-health and its impact on humans by the year 2030 - Blackcoffer Insights'

In [16]:
soup.find('div', class_='tdb-block-inner').text.strip()


''

In [23]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import os
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import re
from textstat import syllable_count

# Download NLTK resources
nltk.download('punkt')
nltk.download('stopwords')

# Function to clean text and remove stop words
def clean_text(text):
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    tokens = word_tokenize(text.lower())  # Tokenize and convert to lowercase
    clean_tokens = [token for token in tokens if token not in stop_words]  # Remove stop words
    return clean_tokens

# Function to calculate sentimental analysis variables
def calculate_sentiment(text):
    positive_words = set(pd.read_csv("positive-words.txt")['positive_words'])
    negative_words = set(pd.read_csv("negative-words.txt")['negative_words'])
    clean_tokens = clean_text(text)
    positive_score = sum(1 for word in clean_tokens if word in positive_words)
    negative_score = sum(1 for word in clean_tokens if word in negative_words)
    polarity_score = (positive_score - negative_score) / (positive_score + negative_score + 0.000001)
    subjectivity_score = (positive_score + negative_score) / (len(clean_tokens) + 0.000001)
    return positive_score, negative_score, polarity_score, subjectivity_score

# Function to calculate readability metrics
def calculate_readability(text):
    sentences = sent_tokenize(text)
    words = clean_text(text)
    total_words = len(words)
    total_sentences = len(sentences)
    avg_sentence_length = total_words / total_sentences
    complex_words = [word for word in words if syllable_count(word) > 2]
    percentage_complex_words = len(complex_words) / total_words
    fog_index = 0.4 * (avg_sentence_length + percentage_complex_words)
    return avg_sentence_length, percentage_complex_words, fog_index

# Function to calculate other metrics
def calculate_other_metrics(text):
    words = clean_text(text)
    total_words = len(words)
    total_sentences = len(sent_tokenize(text))
    average_number_of_words_per_sentence = total_words / total_sentences
    complex_word_count = sum(1 for word in words if syllable_count(word) > 2)
    syllable_per_word = sum(syllable_count(word) for word in words) / total_words
    personal_pronouns = len(re.findall(r'\b(?:i|we|my|ours|us)\b', text, flags=re.IGNORECASE))
    average_word_length = sum(len(word) for word in words) / total_words
    return average_number_of_words_per_sentence, complex_word_count, total_words, syllable_per_word, personal_pronouns, average_word_length

# Read input data
df_input = pd.read_excel("Input.xlsx")

# Create a directory to save extracted text files if it doesn't exist
if not os.path.exists("extracted_files"):
    os.makedirs("extracted_files")

# Function to extract article text from URL and save to file
def extract_and_save_text(url, url_id):
    try:
        # Send a GET request to fetch the webpage content
        response = requests.get(url)
        # Parse the HTML content using BeautifulSoup
        soup = BeautifulSoup(response.content, 'html.parser')
        title = soup.find('title').text.strip()
        # Find the article text
        article_text = soup.find('div', class_='td-post-content tagdiv-type').text.strip()
        
        # Save the extracted article text to a text file
        filename = f"extracted_files/{url_id}.txt"
        with open(filename, "w", encoding="utf-8") as file:
            file.write(article_text)
        print(f"Article extracted and saved: {filename}")
        return filename, title, article_text
    except Exception as e:
        print(f"Error extracting article from {url}: {e}")
        return None

# Iterate through each row in the Input.xlsx file
for index, row in df_input.iterrows():
    url = row['URL']
    url_id = row['URL_ID']
    extract_and_save_text(url, url_id)

# Load stop words
stop_words = set(stopwords.words('english'))

# Iterate through each row and calculate metrics
output_data = []
for index, row in df_input.iterrows():
    url_id = row['URL_ID']
    file_name = f"extracted_files/{url_id}.txt"
    
    # Check if the file exists
    if os.path.exists(file_name):
        # Read text from file
        with open(file_name, 'r', encoding='utf-8') as file:
            text = file.read()
        
        # Perform analysis on the text
        
        # Sentimental Analysis
        positive_score, negative_score, polarity_score, subjectivity_score = calculate_sentiment(text)

        # Analysis of Readability
        avg_sentence_length, percentage_complex_words, fog_index = calculate_readability(text)

        # Other Metrics
        (avg_number_of_words_per_sentence, complex_word_count, word_count,
         syllable_per_word, personal_pronouns, average_word_length) = calculate_other_metrics(text)

        # Append results to output data
        output_data.append([url_id, row['URL'], positive_score, negative_score, polarity_score, subjectivity_score,
                            avg_sentence_length, percentage_complex_words, fog_index, avg_number_of_words_per_sentence,
                            complex_word_count, word_count, syllable_per_word, personal_pronouns, average_word_length])
    else:
        print(f"File not found for URL_ID: {url_id}")

# Create DataFrame for output and save to Excel
output_df = pd.DataFrame(output_data, columns=['URL_ID', 'URL', 'POSITIVE SCORE', 'NEGATIVE SCORE', 'POLARITY SCORE',
                                               'SUBJECTIVITY SCORE', 'AVG SENTENCE LENGTH', 'PERCENTAGE OF COMPLEX WORDS',
                                               'FOG INDEX', 'AVG NUMBER OF WORDS PER SENTENCE', 'COMPLEX WORD COUNT',
                                               'WORD COUNT', 'SYLLABLE PER WORD', 'PERSONAL PRONOUNS', 'AVG WORD LENGTH'])
output_df.to_excel("Output.xlsx", index=False)
print("Output saved successfully.")


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Article extracted and saved: extracted_files/blackassign0001.txt
Article extracted and saved: extracted_files/blackassign0002.txt
Article extracted and saved: extracted_files/blackassign0003.txt
Article extracted and saved: extracted_files/blackassign0004.txt
Article extracted and saved: extracted_files/blackassign0005.txt
Article extracted and saved: extracted_files/blackassign0006.txt
Article extracted and saved: extracted_files/blackassign0007.txt
Article extracted and saved: extracted_files/blackassign0008.txt
Article extracted and saved: extracted_files/blackassign0009.txt
Article extracted and saved: extracted_files/blackassign0010.txt
Article extracted and saved: extracted_files/blackassign0011.txt
Article extracted and saved: extracted_files/blackassign0012.txt
Article extracted and saved: extracted_files/blackassign0013.txt
Error extracting article from https://insights.blackcoffer.com/rise-of-e-health-and-its-imapct-on-humans-by-the-year-2030-2/: 'NoneType' object has no attr

KeyError: 'positive_words'

In [21]:
import os
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from textstat import syllable_count

# Load stop words
stop_words_path = './NLP assignment/StopWords/'
stop_words_files = os.listdir(stop_words_path)
stop_words = set()
for file in stop_words_files:
    with open(os.path.join(stop_words_path, file), 'r') as f:
                stop_words.update(f.read().splitlines())
                
        # Function to remove stop words
        def remove_stop_words(text, stop_words):
            word_tokens = word_tokenize(text)
            filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
            return ' '.join(filtered_text)

        # Function to calculate derived variables
        def calculate_derived_variables(text, positive_words, negative_words, total_words):
            positive_score = sum(1 for word in text if word in positive_words)
            negative_score = sum(1 for word in text if word in negative_words)
            polarity_score = (positive_score - negative_score) / ((positive_score + negative_score) + 0.000001)
            subjectivity_score = (positive_score + negative_score) / (total_words + 0.000001)
            return positive_score, negative_score, polarity_score, subjectivity_score

        # Function to calculate readability metrics
        def calculate_readability(text):
            # Average Number of Words Per Sentence
            sentences = sent_tokenize(text)
            total_words = len(word_tokenize(text))
            total_sentences = len(sentences)
            average_words_per_sentence = total_words / (total_sentences + 0.000001)

            # Complex Word Count and Percentage of Complex Words
            tokenizer = RegexpTokenizer(r'\w+')
            words = tokenizer.tokenize(text)
            complex_words = [word for word in words if syllable_count(word) > 2]
            complex_word_count = len(complex_words)
            percentage_complex_words = (complex_word_count / (total_words + 0.000001)) * 100

            # Fog Index
            fog_index = 0.4 * (average_words_per_sentence + percentage_complex_words)

            # Word Count
            word_count = total_words

            # Syllable Count Per Word
            syllable_count_per_word = sum(syllable_count(word) for word in words) / (total_words + 0.000001)

            # Personal Pronouns Count
            personal_pronouns_count = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE))

            # Average Word Length
            average_word_length = sum(len(word) for word in words) / (total_words + 0.000001)

            return average_words_per_sentence, percentage_complex_words, fog_index, word_count, syllable_count_per_word, \
                   personal_pronouns_count, average_word_length

        # Load positive and negative words
        positive_words_path = './NLP assignment/MasterDictionary/positive-words.txt'
        negative_words_path = './NLP assignment/MasterDictionary/negative-words.txt'
        with open(positive_words_path, 'r') as f:
            positive_words = set(f.read().splitlines())
        with open(negative_words_path, 'r') as f:
            negative_words = set(f.read().splitlines())

        # Initialize sentiment analyzer
        sia = SentimentIntensityAnalyzer()

        # Process each file
        output_data = {'URL_ID': [], 'URL': [], 'POSITIVE SCORE': [], 'NEGATIVE SCORE': [], 'POLARITY SCORE': [],
                       'SUBJECTIVITY SCORE': [], 'AVG SENTENCE LENGTH': [], 'PERCENTAGE OF COMPLEX WORDS': [],
                       'FOG INDEX': [], 'AVG NUMBER OF WORDS PER SENTENCE': [], 'COMPLEX WORD COUNT': [],
                       'WORD COUNT': [], 'SYLLABLE PER WORD': [], 'PERSONAL PRONOUNS': [], 'AVG WORD LENGTH': []}

        articles_folder = './NLP assignment/extracted_articles/'
        for file_name in os.listdir(articles_folder):
            file_name_parts = file_name.split('_')
            if len(file_name_parts) >= 2:
                url_id, url = file_name_parts[0], file_name_parts[1]
                with open(os.path.join(articles_folder, file_name), 'r') as f:
                    article_text = f.read()
                    output_data['URL_ID'].append(url_id)
                    output_data['URL'].append(url)
                    clean_text = ' '.join([word for word in word_tokenize(article_text) if word.lower() not in stop_words])
                    positive_score, negative_score, polarity_score, subjectivity_score = calculate_derived_variables(
                        word_tokenize(clean_text), positive_words, negative_words, len(word_tokenize(clean_text)))
                    average_words_per_sentence, percentage_complex_words, fog_index, word_count, syllable_count_per_word, \
                    personal_pronouns_count, average_word_length = calculate_readability(article_text)
                    output_data['POSITIVE SCORE'].append(positive_score)
                    output_data['NEGATIVE SCORE'].append(negative_score)
                    output_data['POLARITY SCORE'].append(polarity_score)
                    output_data['SUBJECTIVITY SCORE'].append(subjectivity_score)
                    output_data['AVG SENTENCE LENGTH'].append(average_words_per_sentence)
                    output_data['PERCENTAGE OF COMPLEX WORDS'].append(percentage_complex_words)
                    output_data['FOG INDEX'].append(fog_index)
                    output_data['WORD COUNT'].append(word_count)
                    output_data['SYLLABLE PER WORD'].append(syllable_count_per_word)
                    output_data['PERSONAL PRONOUNS'].append(personal_pronouns_count)
                    output_data['AVG WORD LENGTH'].append(average_word_length)

        output_df = pd.DataFrame(output_data)
        output_df.to_excel('output_analysis.xlsx', index=False)
output_df = pd.DataFrame(output_data)
output_df.to_excel('output_analysis.xlsx', index=False)


IndentationError: unindent does not match any outer indentation level (<string>, line 19)

In [37]:
import os
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import RegexpTokenizer
from textstat import syllable_count


# Load stop words
stop_words_path = './NLP assignment/StopWords/'
stop_words_files = os.listdir(stop_words_path)
stop_words = set()
for file in stop_words_files:
    def calculate_readability(text):
        # Average Number of Words Per Sentence
        sentences = sent_tokenize(text)
        total_words = len(word_tokenize(text))
        total_sentences = len(sentences)
        average_words_per_sentence = total_words / (total_sentences + 0.000001)

        # Abstract Average Sentence Length
        abstract_average_sentence_length = len(text) / (total_sentences + 0.000001)

        # Complex Word Count and Percentage of Complex Words
        tokenizer = RegexpTokenizer(r'\w+')
        words = tokenizer.tokenize(text)
        complex_words = [word for word in words if syllable_count(word) > 2]
        
        complex_word_count = len(complex_words)
        percentage_complex_words = (complex_word_count / (total_words + 0.000001)) * 100

        # Fog Index
        fog_index = 0.4 * (average_words_per_sentence + percentage_complex_words)

        # Word Count
        word_count = total_words

        # Syllable Count Per Word
        syllable_count_per_word = sum(syllable_count(word) for word in words) / (total_words + 0.000001)

        # Personal Pronouns Count
        personal_pronouns_count = len(re.findall(r'\b(I|we|my|ours|us)\b', text, re.IGNORECASE))

        # Average Word Length
        average_word_length = sum(len(word) for word in words) / (total_words + 0.000001)

        return average_words_per_sentence, abstract_average_sentence_length, percentage_complex_words, complex_word_count, fog_index, word_count, syllable_count_per_word, personal_pronouns_count, average_word_length

    # Load positive and negative words
    positive_words_path = './NLP assignment/MasterDictionary/positive-words.txt'
    negative_words_path = './NLP assignment/MasterDictionary/negative-words.txt'
    with open(positive_words_path, 'r') as f:
        positive_words = set(f.read().splitlines())
    with open(negative_words_path, 'r') as f:
        negative_words = set(f.read().splitlines())

    # Initialize sentiment analyzer
    sia = SentimentIntensityAnalyzer()

    output_data = {'file_name': [], 'positive': [], 'negative': [], 'polarity_score': [],
                   'subjectivity_score': [], 'average_words_per_sentence': [],
                   'abstract_average_sentence_length': [], 'percentage_complex_words': [],'complex_words_count': [], 'fog_index': [],
                   'word_count': [], 'syllable_count_per_word': [],
                   'personal_pronouns_count': [], 'average_word_length': []}


    articles_folder = './NLP assignment/extracted_articles/'
    for file_name in os.listdir(articles_folder):
        with open(os.path.join(articles_folder, file_name), 'r',  encoding='utf-8' ) as f:
            article_text = f.read()
            # Remove stop words
            clean_text = ' '.join([word for word in word_tokenize(article_text) if word.lower() not in stop_words])
            
            # Calculate sentiment scores
            positive_score, negative_score, polarity_score, subjectivity_score = calculate_derived_variables(word_tokenize(clean_text), positive_words, negative_words, len(word_tokenize(clean_text)))
            
            # Calculate readability metrics
            average_words_per_sentence, abstract_average_sentence_length, percentage_complex_words,  complex_word_count ,fog_index, word_count, syllable_count_per_word, \
            personal_pronouns_count, average_word_length = calculate_readability(article_text)

            output_data['file_name'].append(file_name)
            output_data['positive'].append(positive_score)
            output_data['negative'].append(negative_score)
            output_data['polarity_score'].append(polarity_score)
            output_data['subjectivity_score'].append(subjectivity_score)
            output_data['average_words_per_sentence'].append(average_words_per_sentence)
            output_data['abstract_average_sentence_length'].append(abstract_average_sentence_length)
            output_data['percentage_complex_words'].append(percentage_complex_words)
            output_data['complex_words_count'].append( complex_word_count)
            output_data['fog_index'].append(fog_index)
            output_data['word_count'].append(word_count)
            output_data['syllable_count_per_word'].append(syllable_count_per_word)
            output_data['personal_pronouns_count'].append(personal_pronouns_count)
            output_data['average_word_length'].append(average_word_length)

    # Write output to Excel
output_df = pd.DataFrame(output_data)
output_df.to_excel('output_analyze.xlsx', index=False)
